# **Healthcare Vulnerability Scoring System (HVSS): A Scientific Machine Learning Approach**

## **1. Introduction and Background**

### **1.1 The Healthcare Security Challenge**

Healthcare organizations face unique cybersecurity challenges. Unlike other sectors, security breaches in healthcare can directly impact patient safety, potentially leading to physical harm or even loss of life. Medical devices, electronic health records, and hospital networks all present attractive targets for malicious actors.
The Common Vulnerability Scoring System (CVSS), maintained by the Forum of Incident Response and Security Teams (FIRST), is widely used for assessing the severity of vulnerabilities in software components. CVSS provides a standardized, generic way to communicate vulnerability severity between multiple parties, including manufacturers, hospitals, clinicians, regulators, and researchers. As designed, CVSS (and its Rubric variant) focuses on evaluating individual vulnerabilities in specific software components rather than assessing security risks across complete ecosystems. Additionally, CVSS is not structured to effectively compare pre- and post-mitigation security states, making it challenging to quantify the impact of security controls and mitigations. CVSS relies on linear mathematical formulas which, when faced with multiple metrics and input parameters, create a calculation complexity space that becomes increasingly difficult to maintain accuracy in the final score.

In healthcare environments, where interconnected technologies form complex ecosystems, there's a need for assessment frameworks that can evaluate system-level risks, particularly those that might affect patient safety, sensitive data, and hospital operations, as well as clearly demonstrate the effectiveness of security mitigations.

### **1.2 Overview of HVSS**

The Healthcare Vulnerability Scoring System (HVSS), which originated from the **Edwards Lifesciences Product Security Group**, addresses this need by providing a specialized framework for assessing security vulnerabilities in medical devices residing on healthcare environments. HVSS employs machine learning algorithms that can effectively manage the calculation complexity introduced by multiple metrics and parameters when evaluating complete healthcare ecosystems, overcoming the mathematical challenges of traditional scoring approaches.
HVSS is specifically designed for assessing system-level security risks, allowing for effective comparison between pre- and post-risk mitigation states. This capability enables security teams to quantitatively demonstrate the value of security controls and provide evidence of risk reduction.

The official HVSS calculator lab project is managed on GitHub at: https://github.com/ewprodsec/hvss-calculator-lab, providing an open resource for the healthcare security community. 

HVSS consists of five key components, each designed to capture different aspects of healthcare security:

1. **Exploitability**: Measures the difficulty of executing an attack based on attack vector, complexity, privileges required, and user interaction
2. **XCIA**: Evaluates impacts on Confidentiality, Integrity, and Availability - the traditional security triad
3. **XPS (Patient Safety)**: Assesses potential clinical impact on patients, from negligible effects to life-threatening situations
4. **XSD (Sensitive Data)**: Measures exposure of personal identifiers and the scale of potential data breaches
5. **XHB (Hospital Breach)**: Evaluates potential impact on broader hospital systems beyond the initially compromised device

### **1.3 The Scientific Challenge**

Creating accurate predictive models for HVSS scoring presents several challenges:

- The relationships between various input metrics and final scores are complex and non-linear
- Multiple factors contribute differently to each component score
- Domain expertise must be effectively encoded into machine learning algorithms
- Models must be interpretable for healthcare security professionals

This laboratory experiment applies rigorous scientific methodology to develop machine learning models that accurately predict HVSS scores based on vulnerability characteristics.

---

## **2. Research Methodology**

### **2.1 Experimental Design**

This experiment follows the scientific method:

1. **Research question**: Can machine learning accurately predict healthcare vulnerability scores?
2. **Hypothesis**: Complex algorithms like gradient boosting and neural networks will outperform simple linear models for healthcare vulnerability prediction
3. **Experiment**: Systematic evaluation of multiple algorithms with hyperparameter optimization
4. **Analysis**: Statistical and visual assessment of model performance
5. **Conclusion**: Identification of optimal models for each HVSS component

### **2.2 Data Collection and Processing**

The training data for this experiment comes from expert-assessed vulnerability scenarios, compiled in the "TrainingData.xlsx" file. Each vulnerability has been rated across multiple dimensions and assigned component scores by healthcare security specialists.

Our first step is to import the necessary libraries for data analysis, visualization, and machine learning:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score
import pickle
import time
import os
import json
import uuid
import datetime
from joblib import Parallel, delayed

# Set up visualizations
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("talk")

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning, message="Data Validation extension is not supported and will be removed")
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

This comprehensive set of libraries provides the tools needed for:
- Data manipulation (pandas, numpy)
- Visualization (matplotlib, seaborn)
- Machine learning model development and evaluation (scikit-learn components)
- Parallel processing for computational efficiency (joblib)
- Model persistence for future use (pickle)

---

## **3. Data Acquisition and Preprocessing**

### **3.1 Data Source and Structure**

The HVSS training data is structured across five sheets in the Excel file, each corresponding to a component of the scoring system. We load each sheet into a separate DataFrame for analysis:

In [ ]:
# Define an array of sheets
sheets = ['Exploitability', 'XCIA', 'XPS', 'XSD', 'XHB']

# Create a dictionary to hold the dataframes
dfs = {}

# Loop through the sheets and read contents into separate DataFrames
for sheet in sheets:
    df = pd.read_excel('TrainingData.xlsx', sheet_name=sheet, header=None, skiprows=1)
    
    # Drop rows containing at least one NaN value
    df_cleaned = df.dropna()
    
    dfs[sheet] = df_cleaned

# Let's examine the data characteristics for each sheet
for sheet in sheets:
    print(f"\n{sheet} DataFrame - Shape: {dfs[sheet].shape}")
    print(f"Summary Statistics:")
    print(dfs[sheet].describe())

### **3.2 Data Quality Assessment**

Preprocessing steps include:
- Loading data from each Excel sheet
- Removing incomplete records (rows with missing values)
- Examining data distributions and basic statistics

This initial exploration reveals the shape of our dataset and provides statistical summaries for each HVSS component. Understanding these characteristics is essential before proceeding with model development, as it helps identify potential data quality issues or distribution patterns that might influence model selection.

---

## **4. Exploratory Data Analysis**

### **4.1 Data Structure Examination**

Exploratory Data Analysis (EDA) is a critical scientific step that helps us understand the relationships and patterns within our dataset before applying machine learning algorithms. In this section, we:

1. Properly label the columns in each DataFrame based on their content
2. Visualize correlations between features and target variables
3. Identify potential patterns and relationships that will inform our modeling approach

In [ ]:
# Load the Input sheet for more comprehensive visualizations
input_data = pd.read_excel('TrainingData.xlsx', sheet_name='Input')

for sheet in sheets:
    if sheet == 'Exploitability':
        # Exploitability has 5 columns: AV, EAC, PR, UI, Score
        if dfs[sheet].shape[1] == 5:
            dfs[sheet].columns = ['AV', 'EAC', 'PR', 'UI', 'Exploitability_Score']
    elif sheet == 'XCIA':
        # XCIA has 8 columns: AV, EAC, PR, UI, C, I, A, Score
        if dfs[sheet].shape[1] == 8:
            dfs[sheet].columns = ['AV', 'EAC', 'PR', 'UI', 'C', 'I', 'A', 'Final_XCIA']
    elif sheet == 'XPS':
        # XPS has 6 columns: AV, EAC, PR, UI, XPS, Score
        if dfs[sheet].shape[1] == 6:
            dfs[sheet].columns = ['AV', 'EAC', 'PR', 'UI', 'XPS', 'Final_XPS']
    elif sheet == 'XSD':
        # XSD has 6 columns: AV, EAC, PR, UI, XSD, Score
        if dfs[sheet].shape[1] == 6:
            dfs[sheet].columns = ['AV', 'EAC', 'PR', 'UI', 'XSD', 'Final_XSD']
    elif sheet == 'XHB':
        # XHB has 6 columns: AV, EAC, PR, UI, XHB, Score
        if dfs[sheet].shape[1] == 6:
            dfs[sheet].columns = ['AV', 'EAC', 'PR', 'UI', 'XHB', 'Final_XHB']



### **4.2 Correlation Analysis**

Understanding how different factors correlate with final scores is essential for both model development and interpretation. The `plot_feature_correlations` function creates a color-coded visualization of these relationships:

In [ ]:
def plot_feature_correlations(df, sheet_name):
    # Determine target column based on sheet name
    if sheet_name == 'Exploitability':
        target_col = 'Exploitability_Score'
    else:
        target_col = f'Final_{sheet_name}'
    
    # Check if target column exists
    if target_col not in df.columns:
        print(f"Warning: Target column '{target_col}' not found in {sheet_name}.")
        return
    
    # Calculate correlations with the target
    correlations = df.corr()[target_col].drop(target_col)
    
    # Sort correlations for better visualization
    correlations = correlations.sort_values()  # Ascending order so negative values are at the top
    
    # Create a table-like visualization
    fig, ax = plt.subplots(figsize=(8, len(correlations) * 0.6 + 1))
    
    # Remove axes
    ax.axis('off')
    
    # Create table cells
    table_data = []
    for feature, corr_value in correlations.items():
        table_data.append([feature, f"{corr_value:.2f}"])
    
    # Create the table
    table = ax.table(
        cellText=table_data,
        colLabels=["Feature", "Correlation with " + target_col],
        loc='center',
        cellLoc='center',
        colWidths=[0.4, 0.6]
    )
    
    # Customize table appearance
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.5)
    
    # Color cells based on correlation values
    for i, (feature, corr_value) in enumerate(correlations.items()):
        # Get normalized value between 0 and 1 for colormap
        norm_value = (float(corr_value) + 1) / 2
        
        # Set cell color
        table[(i+1, 1)].set_facecolor(plt.cm.coolwarm(norm_value))
        
        # Set text color (white for dark backgrounds)
        if norm_value < 0.3 or norm_value > 0.7:
            table[(i+1, 1)].get_text().set_color('white')
    
    plt.title(f'Feature Correlations with {target_col} - {sheet_name}', pad=20)
    plt.tight_layout()
    plt.show()

# Visualize each sheet
for sheet in sheets:
    print(f"\n{sheet} DataFrame column names: {list(dfs[sheet].columns)}")
    plot_feature_correlations(dfs[sheet], sheet)

This correlation analysis reveals the strength and direction of relationships between input features and target scores. Strong positive correlations (towards +1.0) indicate that as the feature value increases, the score tends to increase as well. Strong negative correlations (towards -1.0) indicate that as the feature value increases, the score tends to decrease.

### **4.3 Relationship Visualization**

Beyond simple correlations, we need to understand how different components interact. The following code creates scatter plots showing relationships between:
1. Exploitability scores and final component scores
2. Component-specific base scores and final component scores

In [ ]:
for sheet in sheets:
    if sheet != 'Exploitability':
        plt.figure(figsize=(12, 6))
        
        # Left plot: Exploitability Score vs Final Score
        plt.subplot(1, 2, 1)
        plt.scatter(input_data['Exploitability Score'], input_data[f'Final {sheet}'], alpha=0.7)
        
        # Add trend line
        z = np.polyfit(input_data['Exploitability Score'], input_data[f'Final {sheet}'], 1)
        p = np.poly1d(z)
        plt.plot(input_data['Exploitability Score'], p(input_data['Exploitability Score']), 
                "r--", alpha=0.8)
        
        plt.xlabel('Exploitability Score')
        plt.ylabel(f'Final {sheet} Score')
        plt.title(f'Exploitability Score vs Final {sheet}')
        plt.grid(True, alpha=0.3)
        
        # Right plot: Component Score vs Final Score  
        plt.subplot(1, 2, 2)
        plt.scatter(input_data[f'{sheet} Score'], input_data[f'Final {sheet}'], 
                   color='green', alpha=0.7)
        
        # Add trend line
        z = np.polyfit(input_data[f'{sheet} Score'], input_data[f'Final {sheet}'], 1)
        p = np.poly1d(z)
        plt.plot(input_data[f'{sheet} Score'], p(input_data[f'{sheet} Score']), 
                "r--", alpha=0.8)
        
        plt.xlabel(f'{sheet} Base Score')
        plt.ylabel(f'Final {sheet} Score')
        plt.title(f'{sheet} Base Score vs Final {sheet} Score')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

These visualizations reveal whether the relationships between variables are linear, suggesting that linear regression might be effective, or non-linear, indicating that more complex algorithms like random forests or gradient boosting might be necessary.

---

## **5. Feature Engineering and Preparation**

### **5.1 Feature and Target Separation**

Before training our models, we separate the input features (X) from the target variables (y) for each HVSS component:

In [ ]:
# Create dictionaries to hold the features and target variables
X_data = {}
y_data = {}

# Loop through the sheets and split the features and target variables
for sheet in sheets:
    if sheet == 'Exploitability':
        # For Exploitability, features are AV, EAC, PR, UI
        X_data[sheet] = dfs[sheet].iloc[:, :-1].values  # All columns except the last
        y_data[sheet] = dfs[sheet].iloc[:, -1].values   # Only the last column (Exploitability_Score)
    elif sheet == 'XCIA':
        # For XCIA, features are AV, EAC, PR, UI, C, I, A
        X_data[sheet] = dfs[sheet].iloc[:, :-1].values
        y_data[sheet] = dfs[sheet].iloc[:, -1].values
    else:
        # For XPS, XSD, XHB, features are AV, EAC, PR, UI, X
        X_data[sheet] = dfs[sheet].iloc[:, :-1].values
        y_data[sheet] = dfs[sheet].iloc[:, -1].values
    
    print(f"{sheet} - Features shape: {X_data[sheet].shape}, Target shape: {y_data[sheet].shape}")

This critical step:
- Creates separate arrays for input features (X) and target variables (y)
- Accounts for the different structure of each component's data
- Confirms the dimensions of our feature and target arrays

The feature arrays will be used as input to our machine learning models, while the target arrays contain the values we're trying to predict.

---

## **6. Algorithm Selection and Hyperparameter Optimization**

### **6.1 Model Selection Strategy**

In scientific machine learning, selecting the right algorithm for a specific problem is crucial. Rather than making assumptions about which algorithm might work best, we take a systematic, empirical approach by testing multiple algorithms with diverse mathematical foundations:

In [ ]:
# Define the algorithms to test - keeping base definitions simple
base_algorithms = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet': ElasticNet(),
    'SVR (RBF)': SVR(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'AdaBoost': AdaBoostRegressor(),
    'KNeighbors': KNeighborsRegressor(),
    'MLP': MLPRegressor()
}

This approach includes:
- **Linear models**: Linear Regression, Ridge, Lasso, ElasticNet
- **Tree-based models**: Random Forest, Gradient Boosting, AdaBoost
- **Instance-based models**: K-Nearest Neighbors
- **Kernel methods**: Support Vector Regression
- **Neural networks**: Multi-Layer Perceptron

Each algorithm family has different strengths and weaknesses, making them suitable for different types of relationships in the data.

### **6.2 Hyperparameter Optimization**

For each algorithm, we define a grid of potential hyperparameters to test. Hyperparameters control the behavior of learning algorithms and can significantly impact model performance:

In [ ]:
# Define hyperparameter grids for each algorithm type - optimized for efficiency while maintaining accuracy
param_grids = {
    'Linear Regression': {
        'fit_intercept': [True]  # Default is often best, but explicitly defined
    },
    
    'Ridge Regression': {
        'alpha': [0.01, 0.1, 1.0, 10.0],  # Focused range
        'solver': ['auto'],  # Let sklearn choose the best solver
        'random_state': [42]
    },
    
    'Lasso Regression': {
        'alpha': [0.01, 0.1, 1.0, 10.0],  # Focused range
        'random_state': [42]
    },
    
    'ElasticNet': {
        'alpha': [0.01, 0.1, 1.0],  # Reduced options
        'l1_ratio': [0.2, 0.5, 0.8],  # Key values in range
        'random_state': [42]
    },
    
    'SVR (RBF)': {
        'kernel': ['rbf'],  # Only most relevant kernel
        'C': [0.1, 1.0, 10.0],  # Focused range
        'gamma': ['scale', 0.1, 0.01],  # Most important values
    },
    
    'Random Forest': {
        'n_estimators': [100, 200],  # Good defaults
        'max_depth': [None, 15, 30],  # None allows full depth
        'min_samples_split': [2, 5],  # Default and one alternative
        'random_state': [42]
    },
    
    'Gradient Boosting': {
        'n_estimators': [100, 200],  # Reduced options
        'learning_rate': [0.05, 0.1],  # Most common good values
        'max_depth': [3, 5, 7],  # Good range for this algorithm
        'subsample': [0.8, 1.0],  # With and without subsampling
        'random_state': [42]
    },
    
    'AdaBoost': {
        'n_estimators': [50, 100, 200],  # Number of weak learners
        'learning_rate': [0.1, 1.0],  # Contribution of each classifier
        'loss': ['linear', 'square', 'exponential'],  # Loss function
        'random_state': [42]
    },
    
    'KNeighbors': {
        'n_neighbors': [3, 5, 7, 9],  # Number of neighbors to consider
        'weights': ['uniform', 'distance'],  # Equal vs distance-weighted
        'algorithm': ['auto'],  # Let sklearn choose the best algorithm 
        'p': [1, 2]  # Distance metric: Manhattan (p=1) or Euclidean (p=2)
    },
    
    'MLP': {
        'hidden_layer_sizes': [(100,), (200,), (100, 50)],  # Single and two-layer options
        'activation': ['relu', 'tanh'],  # Most performant activations
        'solver': ['adam'],  # Best general-purpose solver
        'alpha': [0.0001, 0.001, 0.01],  # Good regularization range
        'learning_rate': ['adaptive'],  # Usually better than constant
        'max_iter': [10000],  # High value to ensure convergence
        'random_state': [42]
    }
}

### **6.3 Grid Search Implementation**

The `tune_single_algorithm` function performs hyperparameter optimization for a single algorithm on a specific HVSS component:

In [ ]:
# Create a dictionary to store the best models for each algorithm and sheet
best_models = {sheet: {} for sheet in sheets}
best_scores = {sheet: {} for sheet in sheets}

# Function to process a single algorithm for a single component
def tune_single_algorithm(algorithm_name, algorithm, param_grid, X, y, component_name, job_index, total_jobs):
    """Process a single algorithm for a given component with enhanced progress reporting"""
    print(f"Starting job {job_index}/{total_jobs}: {component_name} - {algorithm_name}")
    start_time = time.time()
    
    # If no hyperparameters to tune, just evaluate the base model
    if not param_grid:
        print(f"  {component_name} - {algorithm_name}: Running cross-validation (no parameters to tune)")
        cv_scores = cross_val_score(algorithm, X, y, cv=5, scoring='neg_mean_squared_error')
        mse_scores = -cv_scores
        best_score = np.mean(mse_scores)
        score_std = np.std(mse_scores)
        best_model = algorithm
    else:
        # Create grid search with explicit n_jobs=1 since we're handling parallelism at a higher level
        print(f"  {component_name} - {algorithm_name}: Starting GridSearchCV with {len(param_grid)} parameter combinations")
        
        # Count total parameter combinations
        n_combinations = 1
        for param_name, param_values in param_grid.items():
            n_combinations *= len(param_values)
        
        print(f"  {component_name} - {algorithm_name}: Testing {n_combinations} parameter combinations with 5-fold CV")
        
        grid_search = GridSearchCV(
            algorithm, param_grid, cv=5, 
            scoring='neg_mean_squared_error', n_jobs=1, verbose=1  # Set verbose=1 for more output
        )
        
        # Train the model
        grid_search.fit(X, y)
        
        # Get best model and score
        best_model = grid_search.best_estimator_
        best_score = -grid_search.best_score_
        score_std = grid_search.cv_results_['std_test_score'][grid_search.best_index_]
        
        # Report best parameters
        best_params = grid_search.best_params_
        print(f"  {component_name} - {algorithm_name}: Best parameters: {best_params}")
    
    execution_time = time.time() - start_time
    
    print(f"COMPLETED ({job_index}/{total_jobs}): {component_name} - {algorithm_name} - MSE: {best_score:.4f} ± {score_std:.4f} - Time: {execution_time:.2f}s")
    
    return {
        'component': component_name,
        'algorithm_name': algorithm_name,
        'best_model': best_model,
        'best_score': best_score,
        'score_std': score_std,
        'execution_time': execution_time
    }

This function:
1. Takes an algorithm and hyperparameter grid as input
2. Uses cross-validation to find optimal parameters
3. Reports performance metrics and execution time
4. Returns the best model configuration and its score

### **6.4 Parallel Processing for Computational Efficiency**

To optimize the computational efficiency of our hyperparameter search, we implement parallel processing with a fallback to sequential processing if needed:

In [ ]:
# Try the efficient parallelization approach first, with fallback to sequential
try:
    # Create a list of all jobs to run
    all_jobs = []
    for sheet in sheets:
        for algorithm_name, base_algorithm in base_algorithms.items():
            param_grid = param_grids[algorithm_name]
            all_jobs.append((
                algorithm_name, 
                base_algorithm, 
                param_grid,
                X_data[sheet],
                y_data[sheet],
                sheet
            ))
    
    # Report total jobs
    total_jobs = len(all_jobs)
    num_cpus = os.cpu_count()
    print(f"\nRunning {total_jobs} tuning jobs in parallel using {num_cpus} CPU cores")
    print(f"Progress will be reported for each job as it starts and completes")
    print(f"Each job will also report CV progress for better visibility")
    
    # Add job indices to the jobs
    indexed_jobs = []
    for i, job in enumerate(all_jobs):
        indexed_jobs.append(job + (i+1, total_jobs))
    
    # Run jobs in parallel with enhanced progress reporting
    results = Parallel(n_jobs=-1, verbose=50)(
        delayed(tune_single_algorithm)(*job)
        for job in indexed_jobs
    )
    
    # Organize results by component
    for sheet in sheets:
        best_models[sheet] = {}
        best_scores[sheet] = {}
    
    for result in results:
        component_name = result['component']
        algorithm_name = result['algorithm_name']
        best_models[component_name][algorithm_name] = result['best_model']
        best_scores[component_name][algorithm_name] = result['best_score']
    
    # Identify best algorithm for each component
    for sheet in sheets:
        best_algo = min(best_scores[sheet].items(), key=lambda x: x[1])[0]
        best_score = best_scores[sheet][best_algo]
        print(f"\n▶ Best algorithm for {sheet}: {best_algo} with MSE: {best_score:.4f}")
    
    print("\nEfficient parallel processing completed successfully!")
    
except Exception as e:
    print(f"Error in efficient parallel processing: {e}")
    print("\nFalling back to sequential processing...")
    
    # Sequential fallback
    for sheet in sheets:
        print(f"\nProcessing {sheet} component...")
        best_models[sheet] = {}
        best_scores[sheet] = {}
        
        for i, (algorithm_name, base_algorithm) in enumerate(base_algorithms.items()):
            job_index = i + 1
            start_time = time.time()
            param_grid = param_grids[algorithm_name]
            
            result = tune_single_algorithm(
                algorithm_name, 
                base_algorithm, 
                param_grid,
                X_data[sheet],
                y_data[sheet],
                sheet,
                job_index,
                len(base_algorithms)
            )
            
            best_models[sheet][algorithm_name] = result['best_model']
            best_scores[sheet][algorithm_name] = result['best_score']
        
        # Find best algorithm for this sheet
        best_algo = min(best_scores[sheet].items(), key=lambda x: x[1])[0]
        print(f"\n► Best algorithm for {sheet}: {best_algo} with MSE: {best_scores[sheet][best_algo]:.4f}")
    
    print("\nSequential processing completed!")

This approach:
- Attempts to use parallel processing for efficiency
- Provides a sequential fallback if parallel processing fails
- Reports progress throughout the process
- Identifies the best algorithm for each HVSS component

---

## **7. Optimal Algorithm Identification**

### **7.1 Comparative Algorithm Performance**

After completing the hyperparameter optimization, we identify the most effective algorithm for each HVSS component:

In [ ]:
# Identify the best performing algorithm for each sheet after hyperparameter tuning
true_best_algorithms = {}

for sheet in sheets:
    algo_names = list(best_scores[sheet].keys())
    mse_scores = [best_scores[sheet][algo] for algo in algo_names]
    best_idx = np.argmin(mse_scores)
    true_best_algorithms[sheet] = algo_names[best_idx]
    
    print(f"\nBest algorithm for {sheet} after hyperparameter tuning: {algo_names[best_idx]} (MSE: {mse_scores[best_idx]:.4f})")
    
    # Print the hyperparameters of the best model
    best_model = best_models[sheet][algo_names[best_idx]]
    print(f"Best hyperparameters:")
    for param, value in best_model.get_params().items():
        # Filter out some of the less important parameters for readability
        if not param.startswith('_') and param not in ['feature_names_in_', 'n_features_in_', 'n_iter_', 'random_state']:
            print(f"  {param}: {value}")
    
    # Compare with other algorithms
    print("\nAll tuned algorithms ranked by performance:")
    sorted_algos = sorted(zip(algo_names, mse_scores), key=lambda x: x[1])
    for i, (algo, score) in enumerate(sorted_algos):
        print(f"  {i+1}. {algo}: MSE = {score:.4f}")

This section:
- Identifies the best algorithm for each component based on mean squared error
- Displays the optimal hyperparameters for each selected algorithm
- Ranks all algorithms based on their performance for each component

Understanding which algorithms perform best provides insight into the nature of the relationships within our data. For example, if tree-based methods outperform linear models, it suggests non-linear relationships between features and targets.

---

## **8. Comprehensive Model Evaluation**

### **8.1 Train-Test Split for Final Evaluation**

To ensure our final model evaluation is unbiased, we create a train-test split for each HVSS component:

In [ ]:
# Create train/test splits for final evaluation
X_train = {}
X_test = {}
y_train = {}
y_test = {}

# Split data with stratification where possible (80% train, 20% test)
for sheet in sheets:
    X_train[sheet], X_test[sheet], y_train[sheet], y_test[sheet] = train_test_split(
        X_data[sheet], y_data[sheet], test_size=0.2, random_state=42
    )
    
    print(f"{sheet} - Training set: {X_train[sheet].shape[0]} samples, Test set: {X_test[sheet].shape[0]} samples")

This standard machine learning practice:
- Reserves 20% of the data for testing
- Uses 80% of the data for training
- Ensures consistent splits with a fixed random seed

### **8.2 Multi-Metric Model Assessment**

To thoroughly evaluate our models, we use multiple performance metrics beyond just mean squared error:

In [ ]:
# Define function for comprehensive model evaluation
def evaluate_model_comprehensive(model, X_train, y_train, X_test, y_test):
    """
    Comprehensively evaluate a model using multiple performance metrics
    
    This function implements scientific best practices by:
    1. Training the model on the training data
    2. Making predictions on both training and test data
    3. Calculating multiple complementary performance metrics
    4. Returning both metrics and predictions for further analysis
    
    Parameters:
    -----------
    model : scikit-learn estimator
        The machine learning model to evaluate
    X_train, y_train : array-like
        Training data and labels
    X_test, y_test : array-like
        Test data and labels
        
    Returns:
    --------
    dict
        Dictionary containing multiple evaluation metrics and predictions
    """
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Calculate metrics
    train_mse = mean_squared_error(y_train, y_pred_train)
    test_mse = mean_squared_error(y_test, y_pred_test)
    
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    
    train_ev = explained_variance_score(y_train, y_pred_train)
    test_ev = explained_variance_score(y_test, y_pred_test)
    
    return {
        'train_mse': train_mse,       # Mean Squared Error: Average squared difference between predictions and true values
        'test_mse': test_mse,         # Lower values indicate better fit
        
        'train_mae': train_mae,       # Mean Absolute Error: Average absolute difference between predictions and true values
        'test_mae': test_mae,         # More robust to outliers than MSE
        
        'train_r2': train_r2,         # R²: Proportion of variance explained by the model
        'test_r2': test_r2,           # Values range from 0 to 1, with 1 indicating perfect prediction
        
        'train_ev': train_ev,         # Explained Variance: Similar to R² but not penalized for bias
        'test_ev': test_ev,           # Useful for detecting systematic errors
        
        'y_pred_train': y_pred_train, # Predictions on training data for residual analysis
        'y_pred_test': y_pred_test    # Predictions on test data for error analysis
    }

# Evaluate the best model for each sheet
eval_results = {}

for sheet in sheets:
    print(f"\nEvaluating best model for {sheet}...")
    best_algo = true_best_algorithms[sheet]
    model = best_models[sheet][best_algo]
    
    eval_results[sheet] = evaluate_model_comprehensive(
        model, X_train[sheet], y_train[sheet], X_test[sheet], y_test[sheet]
    )
    
    print(f"  Algorithm: {best_algo}")
    print(f"  Train MSE: {eval_results[sheet]['train_mse']:.4f}")
    print(f"  Test MSE: {eval_results[sheet]['test_mse']:.4f}")
    print(f"  Train MAE: {eval_results[sheet]['train_mae']:.4f}")
    print(f"  Test MAE: {eval_results[sheet]['test_mae']:.4f}")
    print(f"  Train R²: {eval_results[sheet]['train_r2']:.4f}")
    print(f"  Test R²: {eval_results[sheet]['test_r2']:.4f}")
    print(f"  Train Explained Variance: {eval_results[sheet]['train_ev']:.4f}")
    print(f"  Test Explained Variance: {eval_results[sheet]['test_ev']:.4f}")

A scientifically rigorous evaluation requires multiple complementary metrics, as different metrics capture different aspects of model performance:

1. **Mean Squared Error (MSE)**: Measures the average squared difference between predicted and actual values. This metric penalizes larger errors more heavily but is sensitive to outliers.

2. **Mean Absolute Error (MAE)**: Measures the average absolute difference between predicted and actual values. This metric is more robust to outliers than MSE.

3. **R-squared (R²)**: Indicates the proportion of variance in the dependent variable that can be predicted from the independent variables. Values range from 0 to 1, with higher values indicating better fit.

4. **Explained Variance Score**: Similar to R², but doesn't penalize for systematic bias in predictions. Useful for detecting consistent over/under-predictions.

Examining these metrics together provides a more complete picture of model performance than any single metric alone.

## **9. Prediction Accuracy Visualization**

### **9.1 Actual vs. Predicted Values**

Visual examination of model performance is a crucial scientific practice that can reveal patterns that might be missed when only reviewing numerical metrics. The following visualizations show how well our models predict both training and test data:

In these scatter plots:
- Each point represents a vulnerability scenario
- The x-axis shows the actual HVSS score
- The y-axis shows the predicted score from our model
- The red dashed line represents perfect prediction (y=x)
- Points close to the line indicate accurate predictions
- Points far from the line indicate prediction errors

The R² value in each plot title quantifies how well the model explains the variance in the data. An R² of 1.0 would indicate perfect predictions, while 0.0 would indicate no predictive ability.

In [ ]:
# Visualize actual vs predicted values
for sheet in sheets:
    best_algo = true_best_algorithms[sheet]
    
    plt.figure(figsize=(12, 10))
    
    # Training data
    plt.subplot(2, 1, 1)
    plt.scatter(y_train[sheet], eval_results[sheet]['y_pred_train'], alpha=0.7)
    # Add the ideal prediction line (y=x) where predictions equal actual values
    plt.plot([min(y_train[sheet]), max(y_train[sheet])], 
             [min(y_train[sheet]), max(y_train[sheet])], 'r--')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'{sheet} - {best_algo} - Training Data (R² = {eval_results[sheet]["train_r2"]:.4f})')
    plt.grid(True, alpha=0.3)
    
    # Testing data
    plt.subplot(2, 1, 2)
    plt.scatter(y_test[sheet], eval_results[sheet]['y_pred_test'], alpha=0.7)
    # Add the ideal prediction line (y=x) where predictions equal actual values
    plt.plot([min(y_test[sheet]), max(y_test[sheet])], 
             [min(y_test[sheet]), max(y_test[sheet])], 'r--')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'{sheet} - {best_algo} - Testing Data (R² = {eval_results[sheet]["test_r2"]:.4f})')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## **10. Residual Analysis**

### **10.1 Residual Distribution and Patterns**

Residual analysis is a cornerstone of statistical model validation. Residuals (the differences between predicted and actual values) provide insights into model adequacy and potential areas for improvement. In an ideal model:

1. Residuals should be randomly distributed around zero
2. Residuals should show no patterns when plotted against predicted values
3. Residuals should follow a normal distribution

Deviations from these expectations can indicate specific model deficiencies:
- Systematic bias (residual mean not zero)
- Heteroscedasticity (non-constant variance)
- Missing non-linear relationships
- Influence of outliers

The following visualizations examine residual patterns for our best models:

In [ ]:
# Calculate and visualize residuals
for sheet in sheets:
    best_algo = true_best_algorithms[sheet]
    
    # Calculate residuals (actual - predicted)
    train_residuals = y_train[sheet] - eval_results[sheet]['y_pred_train']
    test_residuals = y_test[sheet] - eval_results[sheet]['y_pred_test']
    
    plt.figure(figsize=(12, 10))
    
    # Residuals vs Predicted Values (Training data)
    plt.subplot(2, 2, 1)
    plt.scatter(eval_results[sheet]['y_pred_train'], train_residuals, alpha=0.7)
    plt.axhline(y=0, color='r', linestyle='--')  # Reference line at y=0
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')
    plt.title(f'{sheet} - {best_algo} - Training Residuals')
    plt.grid(True, alpha=0.3)
    
    # Residuals vs Predicted Values (Testing data)
    plt.subplot(2, 2, 2)
    plt.scatter(eval_results[sheet]['y_pred_test'], test_residuals, alpha=0.7)
    plt.axhline(y=0, color='r', linestyle='--')  # Reference line at y=0
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')
    plt.title(f'{sheet} - {best_algo} - Testing Residuals')
    plt.grid(True, alpha=0.3)
    
    # Histogram of training residuals with normal distribution reference
    plt.subplot(2, 2, 3)
    plt.hist(train_residuals, bins=20, alpha=0.7, density=True, label='Residuals')
    # If available, could add normal distribution curve for reference
    plt.xlabel('Residual Value')
    plt.ylabel('Density')
    plt.title('Training Residuals Distribution')
    plt.grid(True, alpha=0.3)
    
    # Histogram of testing residuals with normal distribution reference
    plt.subplot(2, 2, 4)
    plt.hist(test_residuals, bins=20, alpha=0.7, density=True, label='Residuals')
    # If available, could add normal distribution curve for reference
    plt.xlabel('Residual Value')
    plt.ylabel('Density')
    plt.title('Testing Residuals Distribution')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.suptitle(f"Residual Analysis: {sheet} Component", y=1.02, fontsize=16)
    plt.show()

## **11. Feature Importance Analysis**

### **11.1 Identifying Key Vulnerability Factors**

Understanding which features most strongly influence HVSS scores is critical for both scientific insight and practical application. Feature importance analysis helps:

1. **Interpret model decisions**: Reveals the factors that drive predictions, making models more transparent
2. **Guide security practices**: Identifies the most critical factors for healthcare security professionals to focus on
3. **Inform future research**: Highlights areas where additional data collection or feature engineering might be beneficial
4. **Validate domain expertise**: Confirms (or challenges) existing understanding of healthcare security vulnerability factors

Different algorithms calculate feature importance in different ways:
- **Tree-based models** (Random Forest, Gradient Boosting): Measure how much each feature reduces impurity when used in splits
- **Linear models**: Use the magnitude of coefficients (normalized if features have different scales)
- **Other models**: May use permutation importance or SHAP values for feature importance estimation

The following analysis examines feature importance across our best models:

In [ ]:
# Analyze feature importance for the best models
for sheet in sheets:
    best_algo = true_best_algorithms[sheet]
    model = best_models[sheet][best_algo]
    
    print(f"\nFeature importance analysis for {sheet} using {best_algo}:")
    
    # Define feature names based on the sheet with descriptive labels
    if sheet == 'Exploitability':
        feature_names = ['Attack Vector (AV)', 'Extended Attack Complexity (EAC)', 
                         'Privileges Required (PR)', 'User Interaction (UI)']
    elif sheet == 'XCIA':
        feature_names = ['Attack Vector (AV)', 'Extended Attack Complexity (EAC)', 
                         'Privileges Required (PR)', 'User Interaction (UI)',
                         'Confidentiality (C)', 'Integrity (I)', 'Availability (A)']
    else:
        feature_names = ['Attack Vector (AV)', 'Extended Attack Complexity (EAC)', 
                         'Privileges Required (PR)', 'User Interaction (UI)', 
                         sheet]
    
    # Check if the model has feature_importances_ attribute (tree-based models)
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        indices = np.argsort(importances)[::-1]
        
        # Print feature ranking with scientific interpretation
        print("Feature importance ranking (normalized importance scores):")
        for i in range(len(feature_names)):
            feature_idx = indices[i]
            importance = importances[feature_idx]
            print(f"  {i+1}. {feature_names[feature_idx]} ({importance:.4f} or {importance*100:.1f}%)")
        
        # Plot feature importance with enhanced visualization
        plt.figure(figsize=(12, 7))
        plt.title(f'Feature Importance Analysis for {sheet} Component\n({best_algo} Model)', fontsize=14)
        
        # Create horizontal bar chart for better readability with many features
        y_pos = np.arange(len(feature_names))
        plt.barh(y_pos, importances[indices], align='center', 
                alpha=0.8, color='skyblue', edgecolor='navy')
        plt.yticks(y_pos, [feature_names[i] for i in indices])
        plt.xlabel('Relative Importance', fontsize=12)
        plt.xlim(0, max(importances) * 1.1)  # Add some padding
        
        # Add importance values as text
        for i, v in enumerate(importances[indices]):
            plt.text(v + max(importances) * 0.01, i, f"{v:.3f}", 
                    color='navy', va='center', fontweight='bold')
            
        plt.tight_layout()
        plt.grid(axis='x', alpha=0.3)
        plt.show()
    
    # For linear models, check for coef_ attribute
    elif hasattr(model, 'coef_'):
        coefficients = model.coef_
        
        # For 1D coefficient array
        if len(coefficients.shape) == 1:
            # Get absolute values for ranking importance
            abs_coefficients = np.abs(coefficients)
            indices = np.argsort(abs_coefficients)[::-1]
            
            # Print feature ranking with scientific interpretation
            print("Feature coefficients ranking (by absolute magnitude):")
            for i in range(len(feature_names)):
                feature_idx = indices[i]
                coef = coefficients[feature_idx]
                abs_coef = abs_coefficients[feature_idx]
                sign = "+" if coef > 0 else "-"
                print(f"  {i+1}. {feature_names[feature_idx]} ({sign}{abs_coef:.4f}) - {'Increases' if coef > 0 else 'Decreases'} score")
            
            # Plot feature coefficients with enhanced visualization
            plt.figure(figsize=(12, 7))
            plt.title(f'Feature Coefficient Analysis for {sheet} Component\n({best_algo} Model)', fontsize=14)
            
            # Create horizontal bar chart colored by coefficient sign
            y_pos = np.arange(len(feature_names))
            colors = ['green' if c > 0 else 'red' for c in coefficients[indices]]
            plt.barh(y_pos, coefficients[indices], align='center', alpha=0.8, color=colors)
            plt.yticks(y_pos, [feature_names[i] for i in indices])
            plt.xlabel('Coefficient Value (Effect on Score)', fontsize=12)
            
            # Add coefficient values as text
            for i, v in enumerate(coefficients[indices]):
                plt.text(v + (0.01 if v >= 0 else -0.01) * max(abs_coefficients), 
                         i, f"{v:.3f}", 
                         color='black', va='center', ha='left' if v >= 0 else 'right', 
                         fontweight='bold')
            
            plt.axvline(x=0, color='black', linestyle='-', alpha=0.5)  # Add reference line at x=0
            plt.tight_layout()
            plt.grid(alpha=0.3)
            plt.show()
    else:
        print(f"Feature importance not directly available for {best_algo}")
        print("Consider using permutation importance or SHAP values for this model type.")

## **12. Model Persistence and Deployment Strategy**

### **12.1 Scientific Reproducibility Through Model Serialization**

In scientific computing, preserving the trained models is essential for reproducibility, deployment, and future research. The process of saving trained models to disk is known as "model persistence" or "serialization." This practice is fundamental to the scientific method, as it enables other researchers to verify findings and build upon established work.

For the HVSS scoring system, model persistence enables:

1. **Reproducible Science**: Other researchers can verify results without retraining models
2. **Incremental Improvement**: Future work can build upon these models rather than starting from scratch
3. **Knowledge Transfer**: Expertise encoded in the models can be shared across institutions
4. **Longitudinal Analysis**: Models from different time periods can be compared to track evolving security landscapes

### **12.2 Implementation of Model Persistence**

We use the Python pickle library to serialize our models into binary files that can be restored later with all trained parameters intact. Our approach includes storing comprehensive metadata alongside each model to ensure proper documentation and provenance.

### **12.3 Operational Deployment Considerations**

Beyond scientific reproducibility, our model persistence strategy accounts for practical deployment requirements in healthcare security environments:

1. **Version Control**: Each model is saved with timestamp information, facilitating version management in production systems
2. **Performance Documentation**: Key performance metrics are stored with each model, allowing for informed decisions about model usage
3. **Feature Documentation**: Input feature names are preserved to ensure correct application in deployment
4. **Hyperparameter Recording**: All configuration parameters are saved to support future optimization efforts

For operational deployment in healthcare environments, these serialized models can be integrated into:

- **Security Assessment Platforms**: Automating the scoring of newly discovered vulnerabilities
- **Vulnerability Management Systems**: Prioritizing remediation efforts based on predicted impacts
- **Security Testing Frameworks**: Evaluating potential attack vectors during penetration testing
- **Regulatory Compliance Tools**: Supporting HIPAA and other regulatory requirements with standardized risk assessment

The implementation of model persistence ensures both scientific validity and practical utility:

In [ ]:
# Save the best models with detailed metadata
print("\nSaving the best models with metadata...")

# First, ensure the models directory exists
os.makedirs('models', exist_ok=True)

# Generate a unique build ID (5 hex characters)
build_id = uuid.uuid4().hex[:5]

# Get current UTC timestamp
build_date = datetime.datetime.now(datetime.timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z")

# Dictionary to collect all models' metadata
all_models_metadata = {
    'build_date': build_date,
    'build_id': build_id,
    'models': {}
}

for sheet in sheets:
    best_algo = true_best_algorithms[sheet]
    model = best_models[sheet][best_algo]
    
    # Create simplified filename with just the component name
    filename = f'models/{sheet.lower()}_model.pkl'
    
    # Create metadata to store alongside the model
    model_metadata = {
        'component': sheet,
        'algorithm': best_algo,
        'performance_metrics': {
            'test_mse': eval_results[sheet]['test_mse'],
            'test_r2': eval_results[sheet]['test_r2']
        },
        'hyperparameters': model.get_params()
    }
    
    # Add this model's metadata to the collective metadata
    all_models_metadata['models'][sheet] = model_metadata
    
    # Create a dictionary with both model and metadata
    model_package = {
        'model': model,
        'metadata': model_metadata
    }
    
    # Save the model package
    with open(filename, 'wb') as f:
        pickle.dump(model_package, f)
    
    print(f"  Saved {sheet} model ({best_algo}) to {filename}")
    print(f"    Test MSE: {eval_results[sheet]['test_mse']:.4f}")
    print(f"    Test R²: {eval_results[sheet]['test_r2']:.4f}")

# Save the collective metadata to a JSON file
metadata_filename = f'models/hvss_models_metadata.json'
with open(metadata_filename, 'w') as f:
    json.dump(all_models_metadata, f, indent=2)

print(f"\nSaved collective metadata to {metadata_filename}")
print("\nModel persistence completed! These models can now be used for HVSS scoring in production environments.")

## **13. Scientific Conclusions and Healthcare Security Implications**

### **13.1 Summary of Experimental Findings**

This scientific investigation has employed rigorous machine learning methodologies to develop predictive models for the Healthcare Vulnerability Scoring System (HVSS). Through systematic algorithm comparison, hyperparameter optimization, and comprehensive evaluation, we have gained valuable insights into the quantitative assessment of healthcare security vulnerabilities.

Our key experimental findings include:

1. **Algorithm Effectiveness**: We compared 10 distinct machine learning algorithms with diverse mathematical foundations and determined which algorithms perform optimally for each HVSS component. This revealed whether linear or non-linear relationships dominate in healthcare vulnerability assessment.

2. **Predictive Performance**: We achieved strong predictive performance across multiple HVSS components, with R² values demonstrating that machine learning models can reliably predict vulnerability scores based on input metrics.

3. **Feature Importance**: We identified the most influential factors in healthcare vulnerability assessment, providing an empirical basis for prioritizing security efforts in healthcare environments.

4. **Component Predictability**: We discovered variation in the predictability of different HVSS components, offering insight into which aspects of healthcare security are most amenable to algorithmic assessment.

### **13.2 Scientific and Healthcare Security Contributions**

This research advances both the scientific understanding of healthcare security vulnerabilities and provides practical tools for the industry:

**Scientific Contributions:**
- Establishes machine learning as a viable approach for healthcare-specific security assessment
- Quantifies the predictive relationships between attack characteristics and security impacts
- Provides empirical evidence for the relative importance of different vulnerability factors
- Creates a methodological framework for reproducible vulnerability prediction research
- Establishes performance benchmarks for future healthcare security assessment studies

**Healthcare Security Implications:**
- Enables consistent, evidence-based scoring of vulnerabilities specific to healthcare environments
- Supports security practitioners with automated assessment capabilities that account for patient safety
- Highlights critical factors for prioritizing security investments in resource-constrained healthcare settings
- Provides a foundation for standardized security metrics that align with healthcare regulatory requirements
- Bridges the gap between technical vulnerability characteristics and healthcare-specific concerns

### **13.3 Limitations and Methodological Considerations**

As with any scientific investigation, it is important to acknowledge limitations in our approach:

1. **Data Constraints**: The models are only as comprehensive as the training data. As new vulnerability types emerge, model retraining will be necessary.

2. **Subjective Elements**: The original scoring of training examples may contain subjective judgments that influence model outputs.

3. **Static Assessment**: The current models provide point-in-time assessments and do not account for evolving threat landscapes without retraining.

4. **Simplification of Complex Factors**: While the models capture key relationships, some nuances of healthcare environments may not be fully represented.

5. **Domain-Specific Applicability**: Models trained specifically for healthcare may not generalize to other sectors without adaptation.

### **13.4 Future Research Directions**

Building on this scientific investigation, several promising research directions emerge:

**Short-term research opportunities:**
1. **Dataset Expansion**: Incorporating a broader range of healthcare vulnerability scenarios to improve model generalizability
2. **Ensemble Methods**: Investigating whether combinations of multiple algorithms can further improve predictive performance
3. **Feature Engineering**: Developing additional healthcare-specific features that capture nuances of medical environments
4. **Deployment Studies**: Evaluating how these models perform in real-world healthcare security operations

**Medium-term research directions:**
1. **Temporal Analysis**: Studying how vulnerability patterns evolve over time as healthcare technology changes
2. **Causal Inference**: Moving beyond correlation to establish causal relationships between security factors and outcomes
3. **Explainable AI**: Developing more transparent models that provide clearer rationales for security assessments
4. **Transfer Learning**: Exploring how models trained on general security data can be adapted for healthcare-specific contexts

**Long-term scientific goals:**
1. **Predictive Security**: Moving from reactive to predictive security models in healthcare
2. **Integrated Risk Framework**: Combining cyber, physical, and clinical risk assessments in a unified framework
3. **Automated Mitigation Strategy Development**: Using model insights to automatically generate mitigation recommendations
4. **Regulatory Alignment**: Developing scientific foundations for healthcare-specific security regulations

### **13.5 Conclusion**

This scientific investigation demonstrates that machine learning can effectively model the complex relationships in healthcare security vulnerability assessment. By applying rigorous methodologies from data science to the domain of healthcare security, we have created predictive models that balance accuracy with interpretability.

The HVSS machine learning models developed in this research represent a significant step toward more objective, consistent, and efficient security vulnerability assessment in healthcare environments. These models not only advance the scientific understanding of healthcare security factors but also provide practical tools that can help protect sensitive patient data and critical healthcare infrastructure.

As healthcare technology continues to evolve and security threats become increasingly sophisticated, data-driven approaches like those developed in this research will play an essential role in protecting healthcare systems and, ultimately, patient safety.